In [33]:
import os, datetime
import time
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from __future__ import absolute_import
from apache_beam import pvalue
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

In [34]:
class CastAsDateTime(beam.DoFn):
  def process(self, element):
    wData = element
    print(wData["time_stamp"])
    weatherDate = str(time.strftime('%Y-%m-%d', time.localtime(wData["time_stamp"])))
    wData['time_stamp'] = weatherDate
    print(wData)
    return [wData]
           
         
PROJECT_ID = 'uber-fair'

# Project ID is required when using the BQ source
options = {
    'project': PROJECT_ID,
}
opts = beam.pipeline.PipelineOptions(flags=[], **options)

# Create beam pipeline using local runner
p = beam.Pipeline('DirectRunner', options=opts)

sql = 'SELECT time_stamp,id FROM weather_modeled.weather LIMIT 5'
bq_source = beam.io.BigQuerySource(query=sql, use_standard_sql=True)

query_results = p | 'Read from BigQuery' >> beam.io.Read(bq_source)

# write PCollection to log file
query_results | 'Write log 1' >> WriteToText('query_results.txt')

# Par do to the date
date_pcoll = query_results | 'Fix Date parDo' >> beam.ParDo(CastAsDateTime())

#write the output table to txt
date_pcoll | 'Write log 2' >> WriteToText('fixed_date_pcoll.txt')

dataset_id = 'weather_modeled'
table_id = 'weather_DR'
schema_id = 'time_stamp:DATE, id:STRING'

# write PCollection to new BQ table
date_pcoll | 'Write BQ table' >> beam.io.WriteToBigQuery(dataset=dataset_id,
                                                table=table_id,
                                                schema=schema_id,
                                                project=PROJECT_ID,
                                                create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                                                write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE,
                                                batch_size=int(10000000))
result = p.run()
result.wait_until_finish()


1543546374
{u'time_stamp': '2018-11-30', u'id': u'a962c95c-caf1-4aa7-bd16-47235ea56e65'}
1543546374
{u'time_stamp': '2018-11-30', u'id': u'b041d6c2-c850-4c42-a5f7-7a29020d8f44'}
1543546374
{u'time_stamp': '2018-11-30', u'id': u'3463b2f2-6326-4237-bf17-df2301e16e83'}
1543546374
{u'time_stamp': '2018-11-30', u'id': u'0533eb82-e9ac-43a5-beb5-7c1ce3cfd529'}
1543546374
{u'time_stamp': '2018-11-30', u'id': u'd64d75db-87cd-452b-8bc7-6dcaf0a6db1e'}


ERROR:root:Exception at bundle <apache_beam.runners.direct.bundle_factory._Bundle object at 0x7fcdf2602128>, due to an exception.
 Traceback (most recent call last):
  File "/home/jupyter/beam_venv_dir/local/lib/python2.7/site-packages/apache_beam/runners/direct/executor.py", line 341, in call
    finish_state)
  File "/home/jupyter/beam_venv_dir/local/lib/python2.7/site-packages/apache_beam/runners/direct/executor.py", line 381, in attempt_call
    result = evaluator.finish_bundle()
  File "/home/jupyter/beam_venv_dir/local/lib/python2.7/site-packages/apache_beam/runners/direct/transform_evaluator.py", line 577, in finish_bundle
    self.runner.finish()
  File "apache_beam/runners/common.py", line 598, in apache_beam.runners.common.DoFnRunner.finish
    self._invoke_bundle_method(self.do_fn_invoker.invoke_finish_bundle)
  File "apache_beam/runners/common.py", line 589, in apache_beam.runners.common.DoFnRunner._invoke_bundle_method
    self._reraise_augmented(exn)
  File "apache_beam/r

'DONE'